In [26]:
import numpy as np

$q(x) = \sqrt{x_1^2+4} + \sqrt{x_2^2+4}$

In [27]:
def evalf(x): 
    assert type(x) is np.ndarray and len(x) == 2 
    return (np.sqrt((x[0]**2)+4)+np.sqrt((x[1]**2)+4))

In [28]:
def evalg(x):
    assert type(x) is np.ndarray and len(x) == 2  
    return np.array([x[0]/np.sqrt(x[0]**2+4), x[1]/np.sqrt(x[1]**2+4)])

In [29]:
def evalh(x):
    assert type(x) is np.ndarray and len(x) == 2 
    return np.array([[4/(x[0]**2+4)**(3/2), 0],[0, 4/(x[1]**2+4)**(3/2)]])

In [30]:
def evalh_inv(x):
    assert type(x) is np.ndarray and len(x) == 2 
    return np.linalg.inv(evalh(x))

In [31]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma):
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    
    alpha = alpha_start
    pk=-gradf
    while evalf(x+alpha*pk)>evalf(x)+gamma*alpha*np.linalg.multi_dot([gradf,pk]):
        alpha=rho*alpha
    return alpha

In [32]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma):
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0.
    assert type(direction) is np.ndarray and len(direction)==2
    
    alpha = alpha_start
    while evalf(x+alpha*direction)>evalf(x)+gamma*alpha*np.linalg.multi_dot([gradf,direction]):
        alpha=rho*alpha
    return alpha

In [33]:
NEWTON_CONSTANT_STEP_LENGTH = 1
NEWTON_BACKTRACKING_LINE_SEARCH = 2
WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH = 3

In [34]:
def find_minimizer_newton(x, tol, line_search_type, *args):
    assert type(x) is np.ndarray and len(x) == 2 
    assert type(tol) is float and tol>=0 
    g_x = evalg(x)
    
    if(line_search_type == NEWTON_BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]
    
    if line_search_type == NEWTON_CONSTANT_STEP_LENGTH:
        k=0
        while (np.linalg.norm(g_x) > tol):
            D_k = evalh_inv(x)
            direction = -np.matmul(D_k, g_x)
            step_length = 1.0
            x = np.add(x, np.multiply(step_length,direction))
            k += 1 
            g_x = evalg(x) 
    elif line_search_type == NEWTON_BACKTRACKING_LINE_SEARCH:
        k=0
        while (np.linalg.norm(g_x) > tol):
            D_k = evalh_inv(x)
            direction = -np.matmul(D_k, g_x)
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, direction, alpha_start, rho, gamma)
            x = np.add(x, np.multiply(step_length,direction)) 
            k += 1 
            g_x = evalg(x) 
    else:
        raise ValueError("Unknown line search type is entered")
    return x, evalf(x), k

2. [R] Consider η
k = 1, ∀k = 1, 2, . . . in Algorithm 1. With starting point x
0 = (2, 2) and a stopping tolerance
τ = 10−9
, find the number of iterations taken by the Newton’s method. Compare the number of iterations
with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and
minimum objective function value in each case. Comment on your observations

In [39]:
x_start= np.array([2.0,2.0])
tol= 1e-9

alpha_start = 1.0
rho = 0.5
gamma = 0.5

In [40]:
x,y,k = find_minimizer_newton(x_start,tol,NEWTON_CONSTANT_STEP_LENGTH)
print("For newtons method with constant step length ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

KeyboardInterrupt: 

* From above we can see that this method is not terminating because it is keep on oscillating between x = (2,2) and x = (-2,-2) so it is not converging to the optmal value

In [41]:
x,y,k = find_minimizer_newton(x_start,tol,NEWTON_BACKTRACKING_LINE_SEARCH,alpha_start,rho,gamma)
print("For newtons method with backtracking line search ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For newtons method with backtracking line search 
No of iterations taken:  1
Minimizer:  [0. 0.]
Minimum value: 4.0


* In Newton method with bcaktracking line search we got the solution in 1 iteration 

3. [R] Compare the number of iterations obtained for the two variants of Newton’s method in the previous
question, with that of gradient descent algorithm (without scaling) with backtracking line search (implemented
in previous labs), with starting point (2, 2). For backtracking line search, use α
0 = 1, ρ = 0.5, γ = 0.5. Also
compare the minimizer and minimum objective function value in each case. Comment on your observations.

In [42]:
def find_minimizer_gd_wo_scaling(start_x, tol, line_search_type, *args): 
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg(x)
    if(line_search_type == WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]
    k = 0
    while (np.linalg.norm(g_x) > tol): 
        if line_search_type == WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        else:  
            raise ValueError("Unknown line search type is entered")   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 
    return x,evalf(x),k

In [43]:
x,y,k =find_minimizer_gd_wo_scaling(my_start_x, my_tol,WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH, alpha_start,rho,gamma)
print('For Gradient Descent Algorithm with Backtracking line search without scaling :')
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For Gradient Descent Algorithm with Backtracking line search without scaling :
No of iterations taken:  32
Minimizer:  [7.62525638e-10 7.62525638e-10]
Minimum value: 4.0


* We can observe fronm above that gradient descent algorithm with backtracking line search without scaling took 32 iterations which is bad compared to newton method with backtracking line search which just took 1 iteration

4. [R] Consider η
k = 1, ∀k = 1, 2, . . . in Algorithm 1. With starting point x
0 = (8, 8) and a stopping tolerance
τ = 10−9
, find the number of iterations taken by the Newton’s method. Compare the number of iterations
with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and
minimum objective function value in each case. Comment on your observations.

In [47]:
x_start = np.array([8.,8.])
tol= 1e-9

alpha_start = 1.0
rho = 0.5
gamma = 0.5

In [48]:
min_x4,min_obj4,iters4=find_minimizer_newtons(x_start, tol, NEWTON_CONSTANT_STEP_LENGTH)
print('For Newtons method with constant steplength(=1) :')
print()
print('Minimizer value: ',min_x4)
print('Minimum obj fn value: ',min_obj4)
print('Total number of iterations: ',iters4)

<ipython-input-29-da50b4b5d850>:3: RuntimeWarning: overflow encountered in double_scalars
  return np.array([[4/(x[0]**2+4)**(3/2), 0],[0, 4/(x[1]**2+4)**(3/2)]])


LinAlgError: Singular matrix

* From the above we can observe that there is singular matrix error because hessian matrix determinant = 0 and we can't find its inverse so it is terminated

In [49]:
x,y,k = find_minimizer_newton(x_start,tol,NEWTON_BACKTRACKING_LINE_SEARCH,alpha_start,rho,gamma)
print("For newtons method with backtracking line search ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For newtons method with backtracking line search 
No of iterations taken:  13
Minimizer:  [2.83764947e-12 2.83764947e-12]
Minimum value: 4.0


* In newton method with backtracking line search took 13 iterations to obatian optimum value with minimum value of function 4.0

5. [R] Compare the number of iterations obtained for the two variants of Newton’s method in the previous
question, with that of gradient descent algorithm (without scaling) with backtracking line search (implemented
in previous labs), with starting point (8, 8). For backtracking line search, use α
0 = 1, ρ = 0.5, γ = 0.5. Also
compare the minimizer and minimum objective function value in each case. Comment on your observations.

In [51]:
x,y,k=find_minimizer_gd_wo_scaling(x_start, tol,WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH, alpha_start,rho,gamma)
print('For Gradient Descent Algorithm with Backtracking line search without scaling :')
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For Gradient Descent Algorithm with Backtracking line search without scaling :
No of iterations taken:  39
Minimizer:  [8.3177047e-10 8.3177047e-10]
Minimum value: 4.0


* From the above we can see that Gradient descent algorithm with backtracking line search without scaling took 39 iterations to get optimal solution which is more than the newton method with backtracking line search . We can se from the given observations that Newton method is more efficient then the others